In [ ]:
#Import Module
import arcpy

#Set up the workspace - ******change to your workspace here *********
arcpy.env.workspace = r'C:\Users\'

In [ ]:
#referencing the project, map and layout
theProj = arcpy.mp.ArcGISProject("CURRENT")
theMap = theProj.listMaps("Map")[0]
theLyt = theProj.listLayouts("Layout")[0]

In [ ]:
#set up the pdfs directory - ******change to your workspace here *********
pdfDir = r'C:\Users\...'
finalPDF = arcpy.mp.PDFDocumentCreate(pdfDir + "....")

In [ ]:
#listing all layers
images = theMap.listLayers("*")
#filtering which layers are rasters
images = [i for i in images if i.isRasterLayer == True ]
#making sure that only image layers are filtered
print(images)

In [ ]:
#referencing an index shapefile 
index = theMap.listLayers("index")[0]

In [ ]:

#funtion to get the values for the index field
def uniqValues(theTable, theField):
    #empty list to fill in
    uniqValList = []

    #search cursor
    theSearchCur = arcpy.SearchCursor(theTable)

    #loop through the cursor 
    for theRow in theSearchCur:

        #the value from the relevant field
        theVal = theRow.getValue(theField)
        
        #first time through add first value 
        if len(uniqValList) == 0:
            uniqValList.append(theVal)
        #rest of time check if the value is already in the field
        else:
            if not theVal in uniqValList:
                uniqValList.append(theVal)

    #sort the list
    uniqValList.sort()

    #return the list
    return uniqValList

#here we are calling the uniqValues function to return list of districts to loop through
index_unique = uniqValues(index, "index")

In [ ]:


#turning all layers off
for i in images:
    i.visible = False



#chosing a place them looping through the images turning them on and off
#exporting each image to a pdf them to a mapbook
for i in images:
    
    i.visible = True
    
    
    theTitle = theLyt.listElements("TEXT_ELEMENT")[0]
    theTitle.text = ".... " + str(i)
    
    #managing the dataframe
    mf = theLyt.listElements("MAPFRAME_ELEMENT")[0]
    LatDD = 41.750809
    LonDD = -93.706858
    extent = arcpy.Extent(LonDD, LatDD, LonDD, LatDD) 
    mf.panToExtent(extent) 
    mf.camera.setExtent(extent)
    mf.scale = 20000
    
    
    
    #export to a pdf
    print("Creating map for " + str(i))
    exportPDF = pdfDir + str(i) + ".pdf"
    theLyt.exportToPDF(exportPDF)
    
    #add individual map to map book
    finalPDF.appendPages(exportPDF)
    
    i.visible = False

    
#save and close the final pdf file
finalPDF.saveAndClose()


In [ ]:
#choosing an image and looping through the index polygons

#turning all layers off
for i in images:
    i.visible = False


for pol in index_unique:
    
    
    image_1960.visible = True
    
    
    #here we are setting up a clause(query) that will be used to limit what is shown on the map
    #by setting definition query on layer objects
    zoomTo = '"index" = ' + '\'' + pol + '\''


    index.definitionQuery = zoomTo
    

    #zoom to the selected index
    mf = theLyt.listElements('MAPFRAME_ELEMENT','*')[0]
    mf.camera.setExtent(mf.getLayerExtent(index,True,True))


    #get a list of text elements of which there is only one: the Title
    theTitle = theLyt.listElements("TEXT_ELEMENT")[0]
    theTitle.text = "... " + pol 

    #export to a pdf
    print("Creating map for " + pol)
    exportPDF = pdfDir + pol + ".pdf"
    theLyt.exportToPDF(exportPDF)

#cleaning the definition query 
index.definitionQuery = None


In [ ]:
#chosing an index polygon and looping through the historical images

#turning off all layers so that we can turn them on as we need them
for i in images:
    i.visible = False


for i in images:
    
    i.visible = True
    
    theTitle = theLyt.listElements("TEXT_ELEMENT")[0]
    theTitle.text = "...: " + str(i)
    
    #choosing which polygon to use 
    
    zoomTo = '"index" = ' + "'b'"

    
    index.definitionQuery = zoomTo
    
    mf = theLyt.listElements("MAPFRAME_ELEMENT")[0]
    mf.camera.setExtent(mf.getLayerExtent(index, True, True))
    
    
    #export to a pdf
    print("Creating map for " + str(i))
    exportPDF = pdfDir + str(i) + ".pdf"
    theLyt.exportToPDF(exportPDF)
    
    #add individual map to map book
    finalPDF.appendPages(exportPDF)
    
#save and close the final pdf file
finalPDF.saveAndClose()
